* uniform random initialization
* C-shunted qubit : profile aping

In [1]:
from multiprocess import Pool
#from multiprocessing import Pool
from DiSuQ.Torch.optimization import uniformParameters,initializationParallelism
from DiSuQ.Torch import models
from DiSuQ.Torch.optimization import OrderingOptimization,PolynomialOptimization
from DiSuQ.Torch.optimization import loss_Transition
from torch import tensor, float32 as float
from numpy import arange,linspace
from DiSuQ.utils import plotCompare
import pickle
from torch import set_num_threads
set_num_threads(30)

In [2]:
with open('../Examples/target_fluxqubit.p', 'rb') as content: target_info = pickle.load(content)
target_spectrum = target_info['spectrum']
E10 = target_spectrum[:,1] - target_spectrum[:,0]
E20 = target_spectrum[:,2] - target_spectrum[:,0]
target = {'E10':E10[[0,20,-1]],'E20':E20[[0,20,-1]]}

In [3]:
basis = {'O':[3],'I':[6,6],'J':[]}; rep = 'K'
circuit = models.shuntedQubit(basis,sparse=False)

In [4]:
flux_range = linspace(0,1,3,endpoint=True)

In [5]:
flux_profile = [{'I':flux} for flux in flux_range]

In [6]:
lossObjective = loss_Transition(tensor(target['E10'],dtype=float),tensor(target['E20'],dtype=float))

In [7]:
optim = OrderingOptimization(circuit,representation=rep)

In [8]:
parameters = uniformParameters(circuit,3)

In [9]:
circuit.initialization(parameters[0])

In [ ]:
with Pool(5) as multi:
    Search = multi.map(initializationParallelism(optim,lossObjective,flux_profile,10),parameters)

In [13]:
pickle.dump(Search,open( "search.p", "wb" ))

### optimization analysis

In [23]:
Search[0][0]['loss'].iloc[0]

169567.125

In [40]:
Losses = dict()
for index,(parameter,(dLogs,dParams,dCircuit)) in enumerate(zip(parameters,Search)):
    loss = dLogs['loss'].to_numpy()
    if loss[0]-loss[-1] > 0:       
        Losses[str(index)] = loss

In [41]:
plotCompare(arange(10),Losses)